In [1]:
import pandas as pd
import re
import zipfile

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
!ls atusdata/

activity_codes.pdf  atusact_2013.zip    atuscps_2013.zip    atusresp_2013.zip   atusrost_2013.zip   atusrostec_2013.zip atussum_2013.zip    atususersguide.pdf  atuswho_2013.zip    data_dictionary.pdf


##Relevant links:
- [activity_codes.pdf](atusdata/activity_codes.pdf)

- [atususersguide.pdf](atusdata/atususersguide.pdf)

- [data_dictionary.pdf](atusdata/data_dictionary.pdf)

In [32]:
def open_atus_zip(datafile):
    #datafile = 'atussum_2013'
    summary = None
    with zipfile.ZipFile('atusdata/{}.zip'.format(datafile), 'r') as z:
        f = z.open('{}.dat'.format(datafile))
        return pd.read_csv(f)
summary = open_atus_zip('atussum_2013')
summary.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
variables = sorted(list(summary.columns.values)[:24])
meanings =  ['metropolitan_status',
             'education_level',
             'is_hispanic',
             'race',
             'age_edited)',
             'weekly_hours_worked',
             'employment_status',
             'has_multiple_jobs',
             'is_student',
             'school_level',
             'gender',
             'partner_employed',
             'household_children',
             'work_status',
             'weekly_earnings_main',
             'is_holiday',
             'partner_work_status',
             'partner_present',
             'eldercare_minutes',
             'childcare_minutes',
             'youngest_child_age',
             'date',
             'atus_final_weight',
             'atus_case_id']

summary_columns = {key: meanings[i] for i, key in enumerate(variables)}
meanings_unsorted = [summary_columns[key] for key in summary.columns.values[:24]]
meanings_unsorted.extend(summary.columns.values[24:])

In [7]:
# summary = pd.read_csv("atusdata/atussum_2013.dat")
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 36.0 MB


In [8]:
summary.columns

Index(['tucaseid', 'TUFINLWGT', 'TRYHHCHILD', 'TEAGE', 'TESEX', 'PEEDUCA',
       'PTDTRACE', 'PEHSPNON', 'GTMETSTA', 'TELFS', 
       ...
       't181501', 't181599', 't181601', 't181801', 't189999', 't500101',
       't500103', 't500105', 't500106', 't500107'],
      dtype='object', length=413)

Pertinent columns:

* TUFINLWGT - statistical weight of respondent
* TRYHHCHILD - age of youngest child in household
* TEAGE - age of respondent
* TESEX - sex of respondent
* TELFS - working status of respondent
* TRCHILDNUM - number of children in household

In [9]:
summary.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
summary2 = summary.rename(columns=summary_columns)
summary3 = summary2
summary3.columns = pd.MultiIndex.from_tuples(list(zip(meanings_unsorted, summary.columns)))
summary3.head() # .columns.values

,atus_case_id,atus_final_weight,youngest_child_age,age_edited),gender,education_level,race,is_hispanic,metropolitan_status,employment_status,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
import parse_activities as pa
#import importlib
#importlib.reload(pa)
#del pa
pa.atus_codes_list
pa.codes_dict


{'t01': 'Personal Care',
 't0101': 'Sleeping',
 't010101': 'Sleeping',
 't010102': 'Sleeplessness',
 't010199': 'Sleeping, n.e.c.*',
 't0102': 'Grooming',
 't010201': 'Washing, dressing and grooming oneself',
 't010299': 'Grooming, n.e.c.*',
 't0103': 'Health-related Self Care',
 't010301': 'Health-related self care',
 't010399': 'Self care, n.e.c.*',
 't0104': 'Personal Activities',
 't010401': 'Personal/Private activities',
 't010499': 'Personal activities, n.e.c.*',
 't0105': 'Personal Care Emergencies',
 't010501': 'Personal emergencies',
 't010599': 'Personal care emergencies, n.e.c.*',
 't0199': 'Personal Care, n.e.c.*',
 't019999': 'Personal care, n.e.c.*',
 't02': 'Household Activities',
 't0201': 'Housework',
 't020101': 'Interior cleaning',
 't020102': 'Laundry',
 't020103': 'Sewing, repairing, & maintaining textiles',
 't020104': 'Storing interior hh items, inc. food',
 't020199': 'Housework, n.e.c.*',
 't0202': 'Food & Drink Prep., Presentation, & Clean-up',
 't020201': 'Fo

In [12]:
# summary2 = summary.rename(columns=summary_columns)
# summary3 = summary2
# summary3.columns = pd.MultiIndex.from_tuples(list(zip(meanings_unsorted, summary.columns)))
# summary3.head() # .columns.values

In [13]:
adults_crit = summary.TEAGE >= 18
no_children_crit = summary.TRCHILDNUM == 0

In [14]:
adults = summary[adults_crit]
adults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10953 entries, 0 to 11384
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 34.6 MB


In [15]:
people_with_no_children = summary[no_children_crit]
people_with_no_children.info()
len(people_with_no_children)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6481 entries, 2 to 11381
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 20.5 MB


6481

In [16]:
adults_with_no_children = summary[adults_crit & no_children_crit]
adults_with_no_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6481 entries, 2 to 11381
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 20.5 MB


In [17]:
def weighted(df, col):
    """
    Pass in dataframe and a name of column (str) to received statistically weighted number based on TUFINLWGT
    Returns a dataframe with a weighted values column as well as a weighted average
    """
    data = df[['TUFINLWGT', col]]
    data[str(col+'_wgt')] = data.TUFINLWGT * data[col]
    avg = data[str(col+'_wgt')].sum() / data.TUFINLWGT.sum()
    return data, avg

data = adults_with_no_children[['TUFINLWGT', 't120303']]
data.head()
data2, avg = weighted(data, 't120303')
data = data.rename(columns={"TUFINLWGT": "weight", "t120303": "minutes"})

data2.head()
avg

190.25402840855642

In [18]:
data['weighted_minutes'] = data.weight * data.minutes

In [19]:
data.head()

,weight,minutes,weighted_minutes
2,10377056.507734,60,6.226234e+08
3,7731257.992805,65,5.025318e+08
4,4725269.227067,90,4.252742e+08
5,2372791.046351,270,6.406536e+08
6,5671341.270490,244,1.383807e+09


In [20]:
# Minutes on average spent watching TV (unweighted) - DO NOT USE
data.minutes.sum() / len(data)

211.67427866070051

In [21]:
# Minutes on average spent watching TV (weighted)
data.weighted_minutes.sum() / data.weight.sum()

190.25402840855642

In [22]:
def average_minutes(data, activity_code):
    activity_col = "t{}".format(activity_code)
    data = data[['TUFINLWGT', activity_col]]
    data = data.rename(columns={"TUFINLWGT": "weight", activity_col: "minutes"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [23]:
sleeping = average_minutes(adults_with_no_children, "010101")
sleepless = average_minutes(adults_with_no_children, "010102")
sleeping, sleepless, (sleeping + sleepless) / 60 # hours

(521.0183492943637, 4.0328730764962595, 8.7508537061809992)

In [24]:
def activity_columns(data, activity_code):
    """For the activity code given, return all columns that fall under that activity."""
    col_prefix = "t{}".format(activity_code)
    return [column for column in data.columns if re.match(col_prefix, column)]

In [25]:
activity_columns(summary, "0101")

['t010101', 't010102']

In [26]:
def average_minutes2(data, activity_code):
    cols = activity_columns(data, activity_code)
    activity_data = data[cols]
    activity_sums = activity_data.sum(axis=1)
    data = data[['TUFINLWGT']]
    data['minutes'] = activity_sums
    data = data.rename(columns={"TUFINLWGT": "weight"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [27]:
pa.categories

{'t01': 'Personal Care',
 't02': 'Household Activities',
 't03': 'Caring For & Helping Household (HH) Members',
 't04': 'Caring For & Helping Nonhousehold (NonHH) Members',
 't05': 'Work & Work-Related Activities',
 't06': 'Education',
 't07': 'Consumer Purchases',
 't08': 'Professional & Personal Care Services',
 't09': 'Household Services',
 't10': 'Government Services & Civic Obligations',
 't11': 'Eating and Drinking',
 't12': 'Socializing, Relaxing, and Leisure',
 't13': 'Sports, Exercise, and Recreation',
 't14': 'Religious and Spiritual Activities',
 't15': 'Volunteer Activities',
 't16': 'Telephone Calls',
 't18': 'Traveling',
 't50': 'Data Codes'}

In [66]:
average_minutes2(adults_with_no_children, "0101") / 60

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


8.750853706181001

In [29]:
pa.codes_dict['t0101']

'Sleeping'

In [67]:
average_minutes2(adults_with_no_children, "01") / 60

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


9.5541911543273592

In [49]:
# grouping
help(adults_with_no_children.groupby)

Help on method groupby in module pandas.core.generic:

groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False) method of pandas.core.frame.DataFrame instance
    Group series using mapper (dict or key function, apply given function
    to group, return result as series) or by a series of columns
    
    Parameters
    ----------
    by : mapping function / list of functions, dict, Series, or tuple /
        list of column names.
        Called on each element of the object index to determine the groups.
        If a dict or Series is passed, the Series or dict VALUES will be
        used to determine the groups
    axis : int, default 0
    level : int, level name, or sequence of such, default None
        If the axis is a MultiIndex (hierarchical), group by a particular
        level or levels
    as_index : boolean, default True
        For aggregated output, return object with group labels as the
        index. Only relevant for DataFrame inpu

## Joining files

In [33]:
respondents = open_atus_zip('atusresp_2013')
activities = open_atus_zip('atusact_2013')
# respondents = pd.read_csv("atusdata/atusresp_2013/atusresp_2013.dat")
# activities = pd.read_csv("atusdata/atusact_2013/atusact_2013.dat")

In [34]:
respondents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 175 entries, TUCASEID to TXTONHH
dtypes: float64(1), int64(172), object(2)
memory usage: 15.3+ MB


In [35]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215576 entries, 0 to 215575
Data columns (total 31 columns):
TUCASEID        215576 non-null int64
TUACTIVITY_N    215576 non-null int64
TEWHERE         215576 non-null int64
TRTCCTOT_LN     215576 non-null int64
TRTCC_LN        215576 non-null int64
TRTCOC_LN       215576 non-null int64
TRTEC_LN        215576 non-null int64
TRTHH_LN        215576 non-null int64
TRTNOHH_LN      215576 non-null int64
TRTOHH_LN       215576 non-null int64
TRTONHH_LN      215576 non-null int64
TRTO_LN         215576 non-null int64
TRWBELIG        215576 non-null int64
TUACTDUR        215576 non-null int64
TUACTDUR24      215576 non-null int64
TUCC5           215576 non-null int64
TUCC5B          215576 non-null int64
TUCC7           215576 non-null int64
TUCC8           215576 non-null int64
TUCUMDUR        215576 non-null int64
TUCUMDUR24      215576 non-null int64
TUDURSTOP       215576 non-null int64
TUEC24          215576 non-null int64
TUSTARTTIM     

In [36]:
respondents.head()

,TUCASEID,TULINENO,TUYEAR,TUMONTH,TEABSRSN,TEERN,TEERNH1O,TEERNH2,TEERNHRO,TEERNHRY,...,TXSPEMPNOT,TXSPUHRS,TXTCC,TXTCCTOT,TXTCOC,TXTHH,TXTNOHH,TXTO,TXTOHH,TXTONHH
0,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,0,0,0,-1,-1,-1
1,20130101130112,1,2013,1,-1,-1,-1,-1,-1,2,...,0,0,0,0,0,0,-1,0,0,-1
2,20130101130123,1,2013,1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,0,-1,-1,-1,-1,-1
3,20130101130611,1,2013,1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,0,0,-1,-1,-1,-1,-1
4,20130101130616,1,2013,1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,0,0,-1,-1,-1,-1,-1


In [37]:
activities.head()

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,TUDURSTOP,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20130101130004,1,-1,0,0,0,-1,0,0,-1,...,2,-1,04:00:00,12:00:00,1,1,1,10101,101,0
1,20130101130004,2,1,5,5,0,-1,5,5,-1,...,1,-1,12:00:00,12:05:00,11,1,1,110101,1101,0
2,20130101130004,3,1,120,120,0,-1,120,120,-1,...,1,-1,12:05:00,14:05:00,12,3,3,120303,1203,0
3,20130101130004,4,1,0,0,0,-1,0,0,-1,...,2,-1,14:05:00,19:00:00,6,3,1,60301,603,0
4,20130101130004,5,1,30,30,0,-1,30,30,-1,...,1,-1,19:00:00,19:30:00,11,1,1,110101,1101,0


In [38]:
merged = pd.merge(respondents, activities, left_on="TUCASEID", right_on="TUCASEID")
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215576 entries, 0 to 215575
Columns: 205 entries, TUCASEID to TXWHERE
dtypes: float64(1), int64(200), object(4)
memory usage: 338.8+ MB


In [40]:
merged[["TUCASEID", "TUACTIVITY_N", "TXTCOC"]].head()

,TUCASEID,TUACTIVITY_N,TXTCOC
0,20130101130004,1,0
1,20130101130004,2,0
2,20130101130004,3,0
3,20130101130004,4,0
4,20130101130004,5,0


In [41]:
telfs = pd.Series({1: "Employed - at work",
                   2: "Employed - absent",
                   3: "Unemployed - laid off",
                   4: "Unemployed - looking",
                   5: "Not in labor force"})

In [42]:
telfs

1       Employed - at work
2        Employed - absent
3    Unemployed - laid off
4     Unemployed - looking
5       Not in labor force
dtype: object

In [43]:
summary.TELFS.map(telfs).head()

0      Not in labor force
1      Employed - at work
2    Unemployed - looking
3      Employed - at work
4      Employed - at work
Name: TELFS, dtype: object